In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-03"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
15980,2024-03-03,Eua Nba,17:30,Minnesota Timberwolves,Los Angeles Clippers,1.93,1.74,218.5,1.80,2.01,1.5,1.96,1.75,rD3tb9mD,0.518135,0.574713,0.555556,0.497512,0.092847,0.4,0.4,NaN,NaN,212.796,72.588373,0.341117,165.914,40.175101,0.242144,150.810,186.692,167.99,225.77,0.0,0.0,0.0,0.0,0.073215,0.077949,0.080050,-3.68,-0.736,-1.263587,0.722144,0.5,-0.222144,0.51,0.102,7.254902,0.628255,0.6,-0.028255
15981,2024-03-03,Eua Nba,21:00,Cleveland Cavaliers,New York Knicks,1.48,2.41,211.5,1.85,1.96,-4.5,1.90,3.59,6PJi5Qft,0.675676,0.414938,0.540541,0.510204,0.090613,0.8,0.2,NaN,NaN,162.702,25.772368,0.158402,205.788,30.067257,0.146108,183.750,184.576,186.34,216.70,0.0,0.0,0.0,0.0,0.338102,0.040830,0.435341,1.33,0.266,1.804511,0.667893,0.9,0.232107,-0.05,-0.010,-141.000000,0.569891,0.6,0.030109
15982,2024-03-03,Eua Nba,23:30,Phoenix Suns,Oklahoma City Thunder,2.14,1.76,235.5,1.81,2.00,1.5,1.95,1.64,Sv8034Ph,0.467290,0.568182,0.552486,0.500000,0.035472,0.4,0.2,NaN,NaN,190.166,46.792901,0.246063,202.630,17.895142,0.088314,171.492,181.354,182.04,182.04,0.0,0.0,0.0,0.0,0.137795,0.070525,0.122119,2.00,0.400,2.850000,0.712729,0.9,0.187271,-1.52,-0.304,-2.500000,0.588999,0.5,-0.088999
15983,2024-03-03,Itália Liga A,13:30,Brindisi,Cremona,1.68,2.22,155.5,1.86,1.92,-3.5,2.01,2.75,G6ag6w3Q,0.595238,0.450450,0.537634,0.520833,0.045689,0.6,0.2,NaN,NaN,326.550,208.815488,0.639459,173.574,40.177111,0.231470,186.766,193.768,164.78,224.80,0.0,0.0,0.0,0.0,0.195814,0.022448,0.219857,-0.14,-0.028,-24.285714,0.000000,0.0,0.000000,-2.86,-0.572,-2.132867,0.000000,0.0,0.000000
15984,2024-03-03,Itália Liga A,14:00,Scafati,Venezia,2.40,1.58,160.5,1.84,1.94,2.5,2.05,1.36,QukTaytf,0.416667,0.632911,0.543478,0.515464,0.049578,0.8,0.4,NaN,NaN,219.160,164.769148,0.751821,124.784,26.533881,0.212638,151.700,163.084,111.22,94.80,0.0,0.0,0.0,0.0,0.291371,0.037413,0.286161,3.49,0.698,2.005731,0.000000,0.0,0.000000,1.95,0.390,1.487179,0.000000,0.0,0.000000
15985,2024-03-03,Itália Liga A,14:15,Pistoia,Tortona,2.04,1.78,156.5,1.80,1.99,1.5,1.90,1.64,CxoP0HQm,0.490196,0.561798,0.555556,0.502513,0.051994,0.0,0.6,NaN,NaN,264.864,128.425144,0.484872,183.836,100.125945,0.544648,171.390,228.172,263.52,147.92,0.0,0.0,0.0,0.0,0.096255,0.070897,0.103869,-1.52,-0.304,-3.421053,0.000000,0.0,0.000000,-3.28,-0.656,-1.189024,0.000000,0.0,0.000000
15986,2024-03-03,Alemanha Pro A,12:00,Dresden,Jena,2.09,1.69,169.5,1.84,1.86,1.5,1.94,1.54,0lnMYK7A,0.478469,0.591716,0.543478,0.537634,0.070185,0.8,0.6,NaN,NaN,142.210,29.322629,0.206192,121.544,33.822160,0.278271,127.214,120.794,121.26,154.38,0.0,0.0,0.0,0.0,0.149652,0.007644,0.162553,-3.86,-0.772,-1.411917,0.000000,0.0,0.000000,-0.80,-0.160,-4.312500,0.000000,0.0,0.000000
15987,2024-03-03,Argentina Liga A,21:00,Zarate,Obera TC,2.32,1.56,152.5,1.84,1.84,2.5,1.97,1.42,zLwtimx5,0.431034,0.641026,0.543478,0.543478,0.072060,0.6,0.6,NaN,NaN,276.614,149.230495,0.539490,160.164,66.810474,0.417138,188.088,246.316,274.62,182.75,0.0,0.0,0.0,0.0,0.277011,0.000000,0.229445,1.91,0.382,3.455497,0.000000,0.0,0.000000,2.80,0.560,1.000000,0.000000,0.0,0.000000
15988,2024-03-03,Austrália Liga Feminina Wnbl,01:30,Perth Lynx F,Townsville F,1.83,1.91,165.5,1.88,1.86,-1.5,1.94,2.07,xC8tGSs0,0.546448,0.523560,0.531915,0.537634,0.070008,0.6,0.6,NaN,NaN,153.142,57.119170,0.372982,90.174,11.975194,0.132801,129.688,96.214,121.26,109.20,0.0,0.0,0.0,0.0,0.030251,0.007563,0.045847,0.67,0.134,6.194030,0

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Nba,Cleveland Cavaliers,New York Knicks,211.5,1.85,1.0000,Over
1,13:30,Itália Liga A,Brindisi,Cremona,155.5,1.86,1.0000,Over
2,01:30,Austrália Liga Feminina Wnbl,Perth Lynx F,Townsville F,165.5,1.88,1.0000,Over
3,02:00,Coreia Do Sul Kbl,Suwon KT,LG Sakers,164.5,1.82,1.0000,Over
4,04:00,Coreia Do Sul Kbl,Seoul Knights,KCC Egis,160.5,1.83,1.0000,Over
5,14:00,Espanha Leb Plata,Alginet,L'Hospitalet,143.5,1.85,1.0000,Over
6,00:00,Eua Ncaa,San Diego Toreros,Pacific,151.5,1.91,1.0000,Over
7,00:00,Eua Ncaa,St. Marys (CA),Gonzaga,141.5,1.91,1.0000,Over
8,00:00,Eua Ncaa,UC Irvine,Long Beach State,155.5,1.91,1.0000,Over
9,11:15,França Lfb Feminina,Landes F,Lyon F,143.5,1.86,1.0000,Over
